In [1]:
pip install duckdb boto3 rasterio shapely mercantile matplotlib geopandas pyproj


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pyproj
import os
import rasterio
import duckdb
import boto3
from botocore import UNSIGNED
from botocore.config import Config
from rasterio.windows import from_bounds
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.session import AWSSession
from rasterio.warp import calculate_default_transform
from rasterio.crs import CRS
from shapely.geometry import box, mapping
from rasterio.plot import show
import mercantile as merc
from matplotlib import pyplot

In [3]:
#importing buildings from Microsoft building footprints dataset
# Install and load httpfs and spatial extensions for DuckDB
duckdb.sql('INSTALL httpfs')
duckdb.sql('LOAD httpfs')
duckdb.sql('INSTALL spatial')
duckdb.sql('LOAD spatial')

#s3 bucket with the dataset
prefix = "https://data.source.coop/hdx/microsoft-open-buildings/geoparquet-2.0/RegionName=UnitedStates/quadkey=023111311/part-00084-66ec874b-f074-4991-9da4-67716003c6cd.c000.parquet"

totalcount = duckdb.sql(f"SELECT COUNT(*) FROM read_parquet('{prefix}')")

totalcount

#columns = duckdb.sql(f"DESCRIBE SELECT * FROM read_parquet('{prefix}')")
#columns

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       731755 │
└──────────────┘

In [4]:
#minx, miny = -90.394478,38.555146 lowerleft
#  -90.181618,38.555146   #lowerright
#   -90.181618,38.686850 #upperright
# -90.394478,38.686850 #upperleft
#maxx, maxy = -90.181618,38.686850

# Connect to DuckDB (in-memory)
con = duckdb.connect()

# Optional: Install and load spatial extension for advanced geo-clipping
con.execute("INSTALL spatial; LOAD spatial;") 
con.execute("INSTALL httpfs; LOAD httpfs;")

minx, miny = -90.394478,38.555146
maxx, maxy = -90.181618,38.686850

prefix = "https://data.source.coop/hdx/microsoft-open-buildings/geoparquet-2.0/RegionName=UnitedStates/quadkey=023111311/part-00084-66ec874b-f074-4991-9da4-67716003c6cd.c000.parquet"

query = f""" SELECT * FROM read_parquet('{prefix}') WHERE ST_INTERSECTS(geometry, ST_MakeEnvelope({minx},{miny},{maxx},{maxy}))"""


clip_result = con.sql(query)
clip_result.show()

con.execute(f"COPY ({query}) TO 'C:\\Users\\DJ\\Documents\\projects\\spatial_lab_feb_project\\data\\Stl_buildings.parquet' (FORMAT PARQUET)")

#duck_box = duckdb.sql(f"SELECT * FROM read_parquet('{prefix}') WHERE ST_INTERSECTS(geometry, ST_SetSRID(ST_GeomFromText('POLYGON((-90.394478 38.555146, -90.181618 38.555146, -90.181618 38.686850, -90.394478 38.686850, -90.394478 38.555146))'), 26915))");

┌────────────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────┬──────────────┬───────────┐
│       height       │ confidence │                                                                                                                                        geometry                                                                                                                                         │                                 geometry_bbox                                  │  RegionName  │  quadkey  │
│       double       │   double   │                                                                                                                 

In [5]:
# testing Rasterio without manual boto3
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket = 'prd-tnm'
key='StagedProducts/Elevation/13/TIFF/historical/n39w091/USGS_13_n39w091_20240228.tif' #STL Area

session = AWSSession(
    boto3.Session(),
    requester_pays=False,
    aws_unsigned=True
)

response = s3.get_object(Bucket=bucket, Key=key)
data = response["Body"].read()

with rasterio.Env(session):
    with rasterio.open(f"s3://{bucket}/{key}") as src:
        print(src.bounds)

# Define clip area
minx, miny = -90.394478,38.555146
maxx, maxy = -90.181618,38.686850

geom = [mapping(box(minx, miny, maxx, maxy))]


#Clip raster to localized area (TESTING small first) and save locally

with MemoryFile(data) as memfile:
    with memfile.open() as src:
        out_image, out_transform = mask(src, geom, crop=True)
        out_meta = src.meta.copy()

out_meta.update({
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

output_path = r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\clipped_dem_stl.tif'

with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

BoundingBox(left=-91.00055555589358, bottom=37.99944444350717, right=-89.9994444437055, top=39.00055555659458)


In [6]:
# Streaming GeoTIFF from USGS S3 Bucket
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket = 'prd-tnm'
key='StagedProducts/Elevation/13/TIFF/historical/n39w091/USGS_13_n39w091_20240228.tif' #STL Area

response = s3.get_object(Bucket=bucket, Key=key)
data = response["Body"].read()

#Open in Rasterio in Memory
with MemoryFile(data) as memfile:
    with memfile.open() as dataset:

        print(dataset.crs)
        print(dataset.bounds)

# Define clip area
minx, miny = -90.394478,38.555146
maxx, maxy = -90.181618,38.686850

geom = [mapping(box(minx, miny, maxx, maxy))]
stl_bbox = merc.LngLatBbox(minx, miny, maxx, maxy)

#Clip raster to localized area (TESTING small first) and save locally

with MemoryFile(data) as memfile:
    with memfile.open() as src:
        out_image, out_transform = mask(src, geom, crop=True)
        out_meta = src.meta.copy()

out_meta.update({
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

output_path = r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\clipped_dem_stl.tif'



with rasterio.open(output_path, "w", **out_meta) as dest:
    dest.write(out_image)

EPSG:4269
BoundingBox(left=-91.00055555589358, bottom=37.99944444350717, right=-89.9994444437055, top=39.00055555659458)


In [7]:
#Exploring rasterio functions

clipped_dem_stl = rasterio.open(r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\clipped_dem_stl.tif')
#clipped_dem_stl.name

#clipped_dem_stl.read(1) # Reads band information
clipped_dem_stl.crs
clipped_dem_stl.bounds
print(pyproj.datadir.get_data_dir())

c:\Users\DJ\Documents\projects\spatial_lab_feb_project\.venv\Lib\site-packages\pyproj\proj_dir\share\proj


In [8]:
# Reprojecting the raster
from rasterio.enums import Resampling

stl_dem_path = r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\clipped_dem_stl.tif'
reprojected_stl_dem = r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\reproj_dem_stl.tif'
dst_crs = CRS.from_epsg(26915)

# Open source file (ensures a valid DatasetReader) and compute transform/shape for target CRS
with rasterio.open(stl_dem_path) as src:
	transform, width, height = calculate_default_transform(src.crs, dst_crs, src.width, src.height, *src.bounds)
	kwargs = src.meta.copy()
	kwargs.update({
		"crs": dst_crs,
		"transform": transform,
		"width": width,
		"height": height
	})

	# Create destination file and reproject each band
	with rasterio.open(reprojected_stl_dem, "w", **kwargs) as dst:
		for i in range(1, src.count + 1):
			rasterio.warp.reproject(
				source=rasterio.band(src, i),
				destination=rasterio.band(dst, i),
				src_transform=src.transform,
				src_crs=src.crs,
				dst_transform=transform,
				dst_crs=dst_crs,
				resampling=Resampling.bilinear
			)

In [ ]:
# Sampling the Arch pixels
from pyproj import Transformer
reprojected_stl_dem = rasterio.open(r'C:\Users\DJ\Documents\projects\spatial_lab_feb_project\data\reproj_dem_stl.tif')
transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:26915", always_xy=True)
arch_x, arch_y = transformer.transform(-90.1847,38.6247)
print(f"UTM Coordinates: {arch_x}, {arch_y}")
points = [(arch_x,arch_y)]
sample_gen = reprojected_stl_dem.sample(points)
arch_height = next(sample_gen)

print(f"Elevation at Arch: {arch_height}")

# 'Burn' the arch top elevation in
arch_apex_rasterio = reprojected_stl_dem.index(arch_x, arch_y)
print(arch_apex_rasterio)
numpydata = reprojected_stl_dem.read(1)
arch_apex_rasterio = 328

#Update 3x3 area around the center point
row,col = reprojected_stl_dem.index(arch_x, arch_y)
row_start = row - 1
row_end = row + 2
col_start = col - 1
col_end = col + 2
numpydata[(row-1):(row+2), (col-1):(col+2)] = 328

UTM Coordinates: 745086.3297981689, 4278890.776248811
Elevation at Arch: [136.16602]
(790, 2113)


array([[328., 328., 328.],
       [328., 328., 328.],
       [328., 328., 328.]], dtype=float32)